In [1]:
import re
import requests
from bs4 import BeautifulSoup
import unidecode
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
from slugify import slugify
from opencage.geocoder import OpenCageGeocode
import pymongo
from pymongo import MongoClient
from tqdm import tqdm
from tqdm import trange
import folium
from folium import plugins

In [2]:
#creating the Mongo conection
cliente=MongoClient('mongodb://localhost:27017')
db = cliente.gymy
col = db.gymy

In [5]:
gymy = pd.read_csv('gymy_latlong.csv')
gymy.drop(columns = 'Unnamed: 0',inplace=True)

In [14]:
gymy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   names        188 non-null    object 
 1   address      188 non-null    object 
 2   phone        163 non-null    object 
 3   web          117 non-null    object 
 4   description  123 non-null    object 
 5   category     173 non-null    object 
 6   city         173 non-null    object 
 7   link         188 non-null    object 
 8   long         188 non-null    float64
 9   lat          188 non-null    float64
 10  principal    188 non-null    object 
dtypes: float64(2), object(9)
memory usage: 16.3+ KB


In [15]:
lista_points = []
for i in tqdm(range(len(gymy))):
    
    lista_points.append(dict({'type':'Point',
                          'coordinates': [gymy.long[i],
                                         gymy.lat[i]]}))

100%|██████████| 188/188 [00:00<00:00, 7815.73it/s]


In [16]:
gymy['principal'] = pd.Series(lista_points)

In [17]:
type(gymy.head().principal[0])

dict

In [18]:

db.gymy.insert_many(gymy.to_dict('records'))

In [19]:
#creo el índice 2dsphere para poder hacer geoqueries
db.gymy.create_index([('principal', '2dsphere')])

'principal_2dsphere'

In [ ]:
def token(token):
    with open(token, 'r') as f:
        t=f.readlines()[0].split('\n')[0]
    return t


In [43]:
geocoder = OpenCageGeocode(token('geocagetoken.txt'))

In [63]:
prueba = geocoder.geocode('zona 11, guatemala')

In [59]:
prueba[0]['components']

{'ISO_3166-1_alpha-2': 'GT',
 'ISO_3166-1_alpha-3': 'GTM',
 '_category': 'place',
 '_type': 'country',
 'continent': 'North America',
 'country': 'Guatemala',
 'country_code': 'gt'}

In [54]:
ciudad = prueba[0]['components']['city']


'Guatemala City'

In [75]:
#funcion que me permite encontrar puntos cercanos con radio por default de 1000 metros
def find_near(geopoint, radio=1000):
    
    return col.find({'principal':{
        '$near': {'$geometry': geopoint, '$maxDistance':radio}
    }})

In [76]:
#función que permite al usuario elegir una dirección y en base a la misma devuelve los datos necesarios para hacer la geoquery
def getting_coor():
    try:
        x = input('Where would you like your office? ')
        m = 5000
        results = geocoder.geocode(x)
        fcoordinates=[]
        coordinates=[]
        coordinates.append(results[0]['geometry']['lng'])
        coordinates.append(results[0]['geometry']['lat']) 
        bandera = results[0]['annotations']['flag']
        country = results[0]['components']['country']
        city=results[0]['components']['city']
        localidad = country+', '+city
        fcoordinates.append(coordinates)
        fcoordinates.append(m)
        fcoordinates.append(localidad)
        fcoordinates.append(x)
        fcoordinates.append(bandera)
        return fcoordinates
    except:
        return 'Not a valid address'

In [77]:
getting_coor()

'Not a valid address'

In [79]:
#este podría ser el código principal que solicita el input del usuario y devuelve el mapa

coordenadas = getting_coor() #acá inicializa la funcion creada arriba
longlat = coordenadas[0]
latlong = longlat[::-1]
radio = coordenadas[1]
pais = coordenadas[4]
localidad = coordenadas[2]
direccion_solicitud = coordenadas[3]

address={'type': 'Point', 'coordinates':longlat}  # geopoint

ratio=radio  # radio de busqueda

n_ofi=find_near(address, ratio).count()
ofi = find_near(address, ratio)
ofi=pd.DataFrame(ofi)

#creating map object
tooltip = 'Location you chose: {} \n {}'.format(direccion_solicitud,localidad)
mapa=folium.Map(latlong, zoom_start=14, width='100%', height='100%')
datamapa=ofi[['lat', 'long']].values
mapa.add_child(plugins.HeatMap(datamapa, radius=15))
folium.Marker(latlong, tooltip=tooltip, icon=folium.Icon()).add_to(mapa)
mapa.save('MapaFinal.html')
print('Amount of offices near {} are: {}'.format(direccion_solicitud,n_ofi))
mapa

Where would you like your office? zona 14 guatemala


<ipython-input-79-c197b9febaeb>:15: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  n_ofi=find_near(address, ratio).count()


Amount of offices near zona 14 guatemala are: 81


In [30]:
x = getting_coor()

Where would you like your office? zona 14 guatemala


In [31]:
x

[[-90.5184938, 14.5795823],
 5000,
 'Guatemala, Guatemala City',
 'zona 14 guatemala',
 '🇬🇹']

In [36]:
ofi

,_id,names,address,phone,web,description,category,city,link,long,lat,principal
0,5ebdbf30703b9cbb5ec9b0cd,Vitam CrossFit,5a Avenida 12-56 Zona 14,tel:+(502) 2366-6810,https://directorio.guatemala.com/listing_repor...,El mejor y más completo método de acondicionam...,Crossfit,Guatemala - Guatemala,https://www.google.com.ar/maps/place/5+Avenida...,-90.519826,14.584827,"{'type': 'Point', 'coordinates': [-90.51982609..."
1,5ebdbf30703b9cbb5ec9b0bf,Squash Zone,10 Avenida 10-50 Zona 14,(+502) 5580 1015,NaN,NaN,entrenamiento fisico,Guatemala - Guatemala,https://www.google.com.ar/maps/place/Futeca+Sp...,-90.523611,14.582833,"{'type': 'Point', 'coordinates': [-90.5236105,..."
2,5ebdbf30703b9cbb5ec9b0be,Sporta Gym,La Noria,NaN,NaN,NaN,NaN,NaN,https://www.google.com/maps/place/Sporta+La+No...,-90.524555,14.581316,"{'type': 'Point', 'coordinates': [-90.52455479..."
3,5ebdbf30703b9cbb5ec9b03e,Club La Villa,20 Avenida 8-81,(+502) 2333 7324,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/place/Club+la+Vill...,-90.512070,14.580398,"{'type': 'Point', 'coordinates': [-90.51206979..."
4,5ebdbf30703b9cbb5ec9b08c,Micheo Boxing by Futeca Zona 14,10a Avenida 10-50 Zona 14,tel:+(502) 2366-8052,NaN,En Micheo Boxing by Futeca somos una academia ...,Boxeo,Guatemala - Guatemala,"https://www.google.com/maps/place/Futeca,+Cdad...",-90.523159,14.584182,"{'type': 'Point', 'coordinates': [-90.5231594,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
76,5ebdbf30703b9cbb5ec9b05f,Federacion Nacional De Baloncesto,Gimnasio Nacional Teodoro Palacios Flores Z.5,(+502) 2361 9851,NaN,NaN,federaciones deportivas,Guatemala - Guatemala,https://www.google.com/maps/place/Gimnasio+Teo...,-90.518809,14.623166,"{'type': 'Point', 'coordinates': [-90.51880909..."
77,5ebdbf30703b9cbb5ec9b063,Federación Nacional De Esgrima C.D.A.G.,12 Av 25 Cl Zona 5,(+502) 2331 2680,NaN,NaN,federaciones deportivas,Guatemala - Guatemala,https://www.google.com/maps/place/CDAG/@14.623...,-90.521432,14.623381,"{'type': 'Point', 'coordinates': [-90.52143170..."
78,5ebdbf30703b9cbb5ec9b061,Federación Nacional De Boxeo De Guatemala,24 Cl Palacio De Los Deportes 9-31 Zona 4,(+502) 2361 8263,NaN,NaN,federaciones deportivas,Guatemala - Guatemala,https://www.google.com/maps/place/CDAG/@14.623...,-90.521432,14.623381,"{'type': 'Point', 'coordinates': [-90.52143170..."
79,5ebdbf30703b9cbb5ec9b05e,Federación Nacional De Ajedrez De Guatemala,Av 10 A 25-01 Z 5,(+502) 2360 8267,NaN,NaN,deportes,Guatemala - Guatemala,https://www.google.com/maps/place/Federacion+N...,-90.518789,14.623939,"{'type': 'Point', 'coordinates': [-90.5187886,..."


In [42]:
token('geocagetoken.txt')

'd8314354b31d46a298e0f21ffb920a34'